In [146]:
import pandas as pd
import json
from pandas.io.json import json_normalize
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True) 

In [48]:
with open("data/20180704_opendata.swiss.datasets.json") as json_file:
    json_july = json.load(json_file)

## Schema

In [114]:
json_july[0]

{'description': 'Erosionsrisikokarte der landwirtschaftlichen Nutzfläche der Schweiz im 2x2-Meter-Raster auf der Basis des DTM-AV. Die Karte umfasst nur die Tal- und Hügelzone. Gezeigt wird das potentielle, qualitative Erosionsrisiko bei permanenter Ackerbewirtschaftung. Die Gesamtwertung wird einer von drei Gefährdungsstufen (keine Gefährdung, Gefährdung, hohe Gefährdung) zugeordnet, ohne dabei die Nutzung oder die Bewirtschaftungsweise des Bodens zu berücksichtigen. Berechnet wird der langjährige mittlere Bodenabtrag mit der Software AVErosion 1.0 unter Verwendung von MUSLE87 (Modified Universal Soil Loss Equation 1987).',
 'downloads': [{'content_type': 'application/octetstream',
   'created': '2018-07-04T21:01:30.423320',
   'dimensions': {},
   'dl_error': None,
   'file_size': 289821349,
   'filepath': './temp',
   'format': 'ZIP',
   'issued': '2010-11-01T01:00:00',
   'modified': False,
   'path': './temp/Bundesamt für Landwirtschaft BLW/__/a54ffc81-0e56-448f-a99c-2e7477826f6f'

## Analysing the fields

Not all the fields are complete.

From about 20'000 entries we have 257 errors. That's around 1%

In [115]:
data_july = json_normalize(json_july, ['downloads'])
len(data_july[data_july['dl_error'].notna()])

257

In [93]:
data_july['dl_error'].value_counts()

http          251
connection      6
Name: dl_error, dtype: int64

The http errors in detail:
* Mostly "forbidden" (which ones?)
* Then "not found"
* "Unauthorized"

And very few application errors

In [382]:
data_july['status_code'].value_counts()

200.0    23042
403.0      139
404.0       70
401.0       28
500.0       12
502.0        1
400.0        1
Name: status_code, dtype: int64

Detecting the format isn't very complete anymore. We should add:

- PDF
- Different Excel-formats
- JSON
- …

In [117]:
data_july['real_format'].value_counts()

Other                   17934
XLS                      2601
CSV                      1152
tab-separated-values      153
SHP                       148
Image                      86
Name: real_format, dtype: int64

In [221]:
data_july['format'].value_counts()

4634

* The *issued*, *created* and *modified* dates are quite broken.    
* Not every download has a *modified* date (obviously).   
* Not every download has an *issued* date (what does it mean?)
* Downloads "modified" before the starting date of the platform seem to be legit.

In [210]:
modified_dates = data_july['modified'].value_counts()
modified = pd.DataFrame()
modified['date'] = modified_dates.keys()
modified['counts'] = modified_dates.values
modified.loc[0,'date'] = "2000-01-01T00:00:00"
modified.date = pd.to_datetime(modified.date, infer_datetime_format=True)

In [251]:
issued_dates = data_july['issued'].value_counts()
issued = pd.DataFrame()
issued['date'] = issued_dates.keys()
issued['counts'] = issued_dates.values
issued.loc[0,'date'] = "2000-01-01T00:00:00"
issued.date = pd.to_datetime(issued.date, infer_datetime_format=True)

In [372]:
created_dates = data_july['created'].value_counts()
created = pd.DataFrame()
created['date'] = created_dates.keys()
created['counts'] = created_dates.values
created.loc[0,'date'] = "2000-01-01T00:00:00"
created.date = pd.to_datetime(issued.date, infer_datetime_format=True)

In [373]:
mod_plot = [go.Scatter( x=modified.sort_values(by=['date']).date, y=modified.sort_values(by=['date']).counts, name="Modified"),
           go.Scatter( x=issued.sort_values(by=['date']).date, y=issued.sort_values(by=['date']).counts, name="Issued"),
           go.Scatter( x=created.sort_values(by=['date']).date, y=created.sort_values(by=['date']).counts, name="Created")]
iplot(mod_plot)

In [374]:
modified[modified.date < '2013-01-01'].counts.sum()

1170

In [375]:
issued[issued.date < '2013-01-01'].counts.sum()

18564

In [377]:
created[created.date < '2013-01-01'].counts.sum()

647

In [222]:
len(data_july[data_july.modified.notna()])

4634

In [253]:
len(data_july[data_july.issued.notna()])

23299

In [378]:
pd.set_option('display.max_colwidth', -1)
dat = data_july[data_july['modified'].notna()]
dat = dat[dat['modified'] != False]
dat[dat['modified'] > '2020-01-01']

,content_type,created,dimensions,dl_error,file_size,filepath,format,issued,modified,path,real_format,rights,size,status,status_code,total,url
7648,text/csv;charset=UTF-8,2017-07-24T13:46:43.069239,"{'rows': 13, 'columns': 7}",None,2292,./temp,CSV,2013-09-01T02:00:00,2104-10-23T02:00:00,./temp/Schweizerisches Bundesarchiv BAR/federal-consultations-1960-1991/dd6b6fd1-9e8c-47ec-9787-5a83f9a289c7,CSV,NonCommercialAllowed-CommercialAllowed-ReferenceNotRequired,NaN,Analyzed,200.0,91,https://bar-files.opendata.swiss/owncloud/index.php/s/hpjA2U6dpexzysb
7649,text/csv;charset=UTF-8,2017-07-24T13:46:43.069280,"{'rows': 2611, 'columns': 14}",None,267594,./temp,CSV,2013-09-01T02:00:00,2104-10-23T02:00:00,./temp/Schweizerisches Bundesarchiv BAR/federal-consultations-1960-1991/cdd4cf5e-b113-41ee-a60b-1a04691b26d5,CSV,NonCommercialAllowed-CommercialAllowed-ReferenceNotRequired,NaN,Analyzed,200.0,36554,https://bar-files.opendata.swiss/owncloud/index.php/s/kFRaqO7oCRodV5d


In [379]:
pd.set_option('display.max_colwidth', 60)
dat[dat['modified'] < '2008-01-01'].path.head()

650     ./temp/Geoinformation Kanton Zürich/abflussprozesskarte/...
2108    ./temp/Geoinformation Kanton Zürich/arten-der-feuchtgebi...
2109    ./temp/Geoinformation Kanton Zürich/arten-der-trockensta...
3810    ./temp/Bundesamt für Umwelt BAFU/biogeographische-region...
3811    ./temp/Bundesamt für Umwelt BAFU/biogeographische-region...
Name: path, dtype: object

**Rights** are potentially an interesting dimension to visualize

In [99]:
data_july['rights'].value_counts()

NonCommercialAllowed-CommercialWithPermission-ReferenceRequired    18839
NonCommercialAllowed-CommercialAllowed-ReferenceRequired            2636
NonCommercialAllowed-CommercialAllowed-ReferenceNotRequired         1824
Name: rights, dtype: int64

File size grows more or less exponentially

In [330]:
fs = data_july[data_july['file_size'] != "undefined"].sort_values('file_size')
fs_layout = go.Layout(
    xaxis=dict(
        autorange=True
    ),
    yaxis=dict(
        type='log',
        autorange=True
    )
)
fs_data = [go.Scatter(y=fs['file_size'])]
fig = go.Figure(data=fs_data, layout=fs_layout)
iplot(fig)

There are quite a few URLs that are used for many downloads. Non-unique

In [237]:
data_july['url'].value_counts()

https://www.ogd.stadt-zuerich.ch/wms/WMS-ZH-STZH-OGD?                                                                                                                             237
http://wms.geo.admin.ch/?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetCapabilities&lang=de                                                                                                202
http://wmts.geo.admin.ch                                                                                                                                                           37
https://www.ogd.stadt-zuerich.ch/mapproxy/wmts/1.0.0/WMTSCapabilities.xml                                                                                                          33
http://maps.zh.ch                                                                                                                                                                  31
https://www.blw.admin.ch/blw/de/home/politik/datenmanagement/geografisches-informationssys

In [238]:
data_july['status'].value_counts()

Analyzed    22494
Name: status, dtype: int64

In [386]:
len(data_july[data_july['dimensions'] != {}])

4054

In [427]:
total = data_july[data_july.total != "undefined"]
total = total[total.total != "zip exception"]
total = total[total.total != "no download"]
total = total[total.total != "shp exception"]
total = total[total.total != "csv exception"]
total = total[total.total != "Excel exception"]
total = total[total.total != "Multiformat exception"]

total = total[total.total > 0]
total = total.sort_values('total')
len(total)

4113

* Size: The filesize from the portal API
* Filesize: The filesize we calculated
* Total: The number of entries we calculated

***

* Only about 2% of the downloads have a 'size'on the portal
* We could calculate the number of entries for still around 20% of the files from the portal
* The number of entries and the sizes are exponential
* Number of entries (total) and filesize are lightly correlated

In [423]:
size = data_july[data_july['size'].notna()]
size = size[size['size'] > 0]
size = size.sort_values("size")

In [430]:
size_layout = go.Layout(
    xaxis=dict(
        autorange=True
    ),
    yaxis=dict(
        type='log',
        autorange=True
    )
)
size_data = [go.Scatter(y=total['file_size'], name="Filesize"), go.Scatter(y=total['total'], name="Total"), go.Scatter(y=total['size'], name="Size")]
size_fig = go.Figure(data=size_data, layout=size_layout)
iplot(size_fig)

In [369]:
pd.set_option('display.max_colwidth', 60)
size.iloc[366]
size.iloc[307]

content_type                                                       text/csv
created                                          2018-06-15T14:48:57.904029
dimensions                                      {'rows': 24, 'columns': 12}
dl_error                                                               None
file_size                                                              3187
filepath        ./temp/Statistisches Amt Kanton Zürich/wanderungsbilanz-...
format                                                                  CSV
issued                                                  2016-01-21T17:30:35
modified                                                2017-08-31T10:35:05
path            ./temp/Statistisches Amt Kanton Zürich/wanderungsbilanz-...
real_format                                                             CSV
rights             NonCommercialAllowed-CommercialAllowed-ReferenceRequired
size                                                                 643481
status      

About half of the downloads link to a file with `content-type html`

In [431]:
data_july['content_type'].value_counts()

text/html;charset=UTF-8                                                            9297
application/vnd.ms-excel                                                           3320
application/vnd.openxmlformats-officedocument.spreadsheetml.sheet                  2582
text/html; charset=utf-8                                                           2148
application/pdf                                                                    1746
text/csv                                                                            812
application/zip                                                                     412
application/octet-stream                                                            378
application/json                                                                    318
text/xml; charset=UTF-8                                                             290
text/html                                                                           257
application/json; charset=utf-8 

In [436]:
cont_type = data_july[data_july['content_type'].notna()]
len(cont_type[cont_type['content_type'].str.contains('html')])

11748

In [108]:
arr = []
for dataset in json_july:
    if 'downloads' in dataset:
        for dl in dataset['downloads']:
            arr.append({
                'description': dataset["description"],
                'name': dataset['name'],
                'id': dataset['id'],
                'error': dl['dl_error'],
                'size': None if dl['file_size'] == "undefined" else dl['file_size'],
                'rights': dl['rights'],
                'url'
                
            })